# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from Vacation_API import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
city_weather_df = pd.read_csv('../WeatherPy/UpgradedCityWeather.csv')
city_weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max_Temps,Humidity,Cloudiness,Wind_Speed,Country,Date,Max_Temps_in_Fahrenheit,Imperial_Wind_Speed
0,0,Mackay,-21.1500,149.2000,297.62,35,4,4.12,AU,1623981388,76.05,9.23
1,1,Tromsø,69.6496,18.9570,282.51,93,75,0.00,NO,1623981391,48.85,0.00
2,2,Mataura,-46.1927,168.8643,281.19,75,7,3.24,NZ,1623981393,46.47,7.26
3,3,Chapleau,47.8405,-83.4034,295.20,49,75,3.09,CA,1623981396,71.69,6.92
4,4,Longyearbyen,78.2186,15.6401,277.06,65,75,5.66,SJ,1623981399,39.04,12.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Storing latitude and longitude in locations
locations = city_weather_df[['Latitude', 'Longitude']]

humidity = city_weather_df['Humidity']

#Plot Heatmap
fig = gmaps.figure()

#Creating heat leayer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=100, point_radius=1)

#add layer
fig.add_layer(heat_layer)

#Display Figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
city_weather_df[(city_weather_df['Max_Temps_in_Fahrenheit'])]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
